In [1]:
from model import *
from data2 import *
from nets.MaxPoolingWithIndices2D import MaxPoolingWithIndices2D
from nets.MaxUnpoolingWithIndices2D import MaxUnpoolingWithIndices2D

In [2]:
def aspp_block(x, num_filters, rate_scale=1):
    x1 = Conv2D(num_filters, (3, 3), dilation_rate=(3 * rate_scale, 3 * rate_scale), padding="same")(x)
    x1 = BatchNormalization()(x1)

    x2 = Conv2D(num_filters, (3, 3), dilation_rate=(12 * rate_scale, 12 * rate_scale), padding="same")(x)
    x2 = BatchNormalization()(x2)

    x3 = Conv2D(num_filters, (3, 3), dilation_rate=(18 * rate_scale, 18 * rate_scale), padding="same")(x)
    x3 = BatchNormalization()(x3)

    x4 = Conv2D(num_filters, (3, 3), padding="same")(x)
    x4 = BatchNormalization()(x4)

    y = Add()([x1,x2,x3,x4])#x1,x4
    y = Conv2D(num_filters, (1, 1), padding="same")(x1)#104曾的(y改為x1)
    return y 

def ssnet_correct(pretrained_weights = None,num_classes = 1,input_size = (256,256,1)):##這個
    inputs = tf.keras.Input(shape=input_size)
    x = layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    x2, mask_1 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x)
    
    x2 = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x2)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)
    x2 = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x2)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)    
    x3, mask_2 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x2)
    
    x3 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x3)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Activation('relu')(x3)
    x3 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x3)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Activation('relu')(x3)
    x3 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x3)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Activation('relu')(x3)
    x4, mask_3 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x3)
    x4 = layers.Dropout(0.5)(x4)
    
    x4 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x4)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.Activation('relu')(x4)
    x4 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x4)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.Activation('relu')(x4)
    x4 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x4)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.Activation('relu')(x4)
    x5, mask_4 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x4)
    x5 = layers.Dropout(0.5)(x5)
    
    x5 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation('relu')(x5)
    x5 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation('relu')(x5)
    x5 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation('relu')(x5)
    x5, mask_5 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x5)
    x5 = layers.Dropout(0.5)(x5)
    
    # decoder
    x6 = MaxUnpoolingWithIndices2D((2, 2))([x5, mask_5])
    x6 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.Activation('relu')(x6)
    x6 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.Activation('relu')(x6)
    x6 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.Activation('relu')(x6)
    x6 = layers.Dropout(0.5)(x6)

    x7 = MaxUnpoolingWithIndices2D((2, 2))([x6, mask_4])
    x7 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.Activation('relu')(x7)
    x7 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.Activation('relu')(x7)
    x7 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.Activation('relu')(x7)
    x7 = layers.Dropout(0.5)(x7)

    x8 = MaxUnpoolingWithIndices2D((2, 2))([x7, mask_3])
    x8 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.Activation('relu')(x8)
    x8 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.Activation('relu')(x8)
    x8 = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.Activation('relu')(x8)
    x8 = layers.Dropout(0.5)(x8)

    x9 = MaxUnpoolingWithIndices2D((2, 2))([x8, mask_2])
    x9 = layers.Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x9)
    x9 = layers.BatchNormalization()(x9)
    x9 = layers.Activation('relu')(x9)
    x9 = layers.Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x9)
    x9 = layers.BatchNormalization()(x9)
    x9 = layers.Activation('relu')(x9)

    x10 = MaxUnpoolingWithIndices2D((2, 2))([x9, mask_1])
    x10 = layers.Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x10)
    x10 = layers.BatchNormalization()(x10)
    x10 = layers.Activation('relu')(x10)
    
    #newinput =  layers.concatenate([conv9,tempcon9], axis = 3)
    x_1 = layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x10)
    x_1 = layers.BatchNormalization()(x_1)
    x_1 = layers.Activation('relu')(x_1)
    x_1 = layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x_1)
    x_1 = layers.BatchNormalization()(x_1)
    x_1 = layers.Activation('relu')(x_1)    
    x2_1, mask_1 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x_1)
    
    merge2 = layers.concatenate([x2_1,x9], axis = 3)
    x2_1 = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(merge2)
    x2_1 = layers.BatchNormalization()(x2_1)
    x2_1 = layers.Activation('relu')(x2_1)
    x2_1 = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x2_1)
    x2_1 = layers.BatchNormalization()(x2_1)
    x2_1 = layers.Activation('relu')(x2_1)    
    x3_1, mask_2 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x2_1)
    
    merge3 = layers.concatenate([x3_1,x8], axis = 3)
    x3_1 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(merge3)
    x3_1 = layers.BatchNormalization()(x3_1)
    x3_1 = layers.Activation('relu')(x3_1)
    x3_1 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x3_1)
    x3_1 = layers.BatchNormalization()(x3_1)
    x3_1 = layers.Activation('relu')(x3_1)
    x3_1 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x3_1)
    x3_1 = layers.BatchNormalization()(x3_1)
    x3_1 = layers.Activation('relu')(x3_1)
    x4_1, mask_3 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x3_1)
    x4_1 = layers.Dropout(0.5)(x4_1)
    
    merge4 = layers.concatenate([x4_1,x7], axis = 3)
    x4_1 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(merge4)
    x4_1 = layers.BatchNormalization()(x4_1)
    x4_1 = layers.Activation('relu')(x4_1)
    x4_1 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x4_1)
    x4_1 = layers.BatchNormalization()(x4_1)
    x4_1 = layers.Activation('relu')(x4_1)
    x4_1 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x4_1)
    x4_1 = layers.BatchNormalization()(x4_1)
    x4_1 = layers.Activation('relu')(x4_1)
    x5_1, mask_4 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x4_1)
    x5_1 = layers.Dropout(0.5)(x5_1)
    
    merge5 = layers.concatenate([x5_1,x6], axis = 3)
    x5_1 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(merge5)
    x5_1 = layers.BatchNormalization()(x5_1)
    x5_1 = layers.Activation('relu')(x5_1)
    x5_1 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x5_1)
    x5_1 = layers.BatchNormalization()(x5_1)
    x5_1 = layers.Activation('relu')(x5_1)
    x5_1 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x5_1)
    x5_1 = layers.BatchNormalization()(x5_1)
    x5_1 = layers.Activation('relu')(x5_1)
    x5_1, mask_5 = MaxPoolingWithIndices2D(pool_size=(2, 2))(x5_1)
    x5_1 = layers.Dropout(0.5)(x5_1)
    
    # decoder
    x6 = MaxUnpoolingWithIndices2D((2, 2))([x5_1, mask_5])
    x6 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.Activation('relu')(x6)
    x6 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.Activation('relu')(x6)
    x6 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.Activation('relu')(x6)
    x6 = layers.Dropout(0.5)(x6)

    x7 = MaxUnpoolingWithIndices2D((2, 2))([x6, mask_4])
    x7 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.Activation('relu')(x7)
    x7 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.Activation('relu')(x7)
    x7 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.Activation('relu')(x7)
    x7 = layers.Dropout(0.5)(x7)

    x8 = MaxUnpoolingWithIndices2D((2, 2))([x7, mask_3])
    x8 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.Activation('relu')(x8)
    x8 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.Activation('relu')(x8)
    x8 = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.Activation('relu')(x8)
    x8 = layers.Dropout(0.5)(x8)

    x9 = MaxUnpoolingWithIndices2D((2, 2))([x8, mask_2])
    x9 = layers.Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_uniform')(x9)
    x9 = layers.BatchNormalization()(x9)
    x9 = layers.Activation('relu')(x9)
    x9 = layers.Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x9)
    x9 = layers.BatchNormalization()(x9)
    x9 = layers.Activation('relu')(x9)

    x10 = MaxUnpoolingWithIndices2D((2, 2))([x9, mask_1])
    x10 = layers.Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_uniform')(x10)
    x10 = layers.BatchNormalization()(x10)
    x10 = layers.Activation('relu')(x10)

   
    x10 = aspp_block(x10, 64)
    #xoutputs = layers.Conv2D(num_classes, (1, 1), padding='valid',activation='sigmoid', kernel_initializer='he_uniform')(x10)
    xoutputs = layers.Conv2D(num_classes, (1, 1), padding='valid',activation=relu_advanced, kernel_initializer='he_uniform')(x10)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #xoutputs = Conv2D(1, 1, activation = 'sigmoid')(xoutputs)
    #conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    #conv10 = Conv2D(1, 1)(conv9)
    
    model = tf.keras.Model(inputs = inputs, outputs = xoutputs)
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [dice_coef_loss], metrics = 'accuracy')
    import os 
    os.environ['KERAS_BACKEND']='tensorflow'
    import tensorflow.python.keras.backend as K
    smooth =1
    def dice_coef(y_true, y_pred, smooth=1):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    def dice_loss(y_true, y_pred):
        loss = 1 - dice_coef(y_true, y_pred)
        return loss
    
    def dice_coef_loss(y_true, y_pred):
        return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    def FocalTverskyLoss(targets, inputs, alpha=ALPHA, beta=BETA, gamma=GAMMA, smooth=1e-6):
    
        #flatten label and prediction tensors
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)
        
        #True Positives, False Positives & False Negatives
        TP = K.sum((inputs * targets))
        FP = K.sum(((1-targets) * inputs))
        FN = K.sum((targets * (1-inputs)))
               
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        FocalTversky = K.pow((1 - Tversky), gamma)
        
        return FocalTversky
    def dice_coef(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    def dice_coefloss(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return -(2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    def iou_coef(y_true, y_pred, smooth=1):
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
        union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
        iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
        return iou
    def iou_coef_only(y_true, y_pred, smooth=1):
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
        union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
        iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
        return -iou
    def connectloss(y_true, y_pred):
        return -(0.4*dice_coef(y_true, y_pred)+0.6*iou_coef(y_true, y_pred))

    #model.compile(optimizer = Adam(lr = 1e-6), loss = 'binary_crossentropy', metrics = 'accuracy')
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [dice_coef_loss], metrics = 'accuracy')
    #model.compile(loss='mse',optimizer=Adam(lr = 1e-5),metrics='metric_fun')
    model.compile(optimizer = Adam(lr = 1e-4), loss = [connectloss], metrics=[dice_coef, iou_coef,'accuracy'])
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [dice_coef_loss], metrics = 'accuracy')
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [FocalTverskyLoss], metrics = 'accuracy')
    #model.compile(optimizer = sgd(lr = 1e-6), loss = [dice_loss], metrics = 'accuracy')
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [dice_loss], metrics = 'accuracy')binary_accuracy、dice_coef
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [dice_loss], metrics = 'binary_accuracy')
    #model.compile(optimizer = Adam(lr = 1e-6), loss = [categorical_crossentropy], metrics = 'categorical_crossentropy')
    if(pretrained_weights):
    	model= load_model(pretrained_weights)
    
    return model

In [3]:
imgs_train,imgs_mask_train = geneTrainNpy("D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256","D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256")

['D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_0.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_1.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_100.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_1000.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10000.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10001.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10002.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10003.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10004.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\\image_10005.png', 'D:/Alldata/ddsm_hospital_inbreast/croponly/train/traincomtight256\

In [ ]:
import datetime
log_dir = "logs1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
mycallback = [tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
              tf.keras.callbacks.EarlyStopping(patience=20, monitor = 'val_loss'),
              ModelCheckpoint('sci_modify4.hdf5', monitor='val_loss',verbose=1, save_weights_only=True, save_best_only=True)]
#myGene = train2numpy(2,'data/membrane_hospital_aughand/train','image','label',data_gen_args1,save_to_dir = None)
#valdata_image,valdata_mask = valGenerator(2,'data/membrane_hospital_aughand/val','image','label',data_gen_args1)
#model = unet3("test_hope2.hdf5")
#model = usnet2(None,1,(512,512,1))
#model = get_wnet_fit(None,1,(256,256,1))
model = ssnet_correct(None,1,(256,256,1))
#model = get_wnet_fit_vgg16(None,1,(256,256,1))
model.summary()
history=model.fit(imgs_train, imgs_mask_train,batch_size=4,epochs=5000,verbose=1,validation_split=0.2, shuffle=True, callbacks=[mycallback])

In [ ]:
tf.keras.utils.plot_model(model, to_file='Model2.png')

In [ ]:
testGene = testGenerator("data/membrane_hospital_aughand_fakeadd/test/tight256_clahe/")#cropcom、membrane_hospital_aughand_fake、membrane_hospital_aughand_ddsm、membrane_hospital_aughand_inbreast
model = ssnet_correct()
#model.summary()
model.load_weights("test.hdf5")


results = model.predict_generator(testGene,48,verbose=1)
saveResult("data/membrane_hospital_aughand_fakeadd/test/pred/",results)

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

gt_path='data/membrane_hospital/test/tight256labelpng/'


files=sorted(os.listdir(gt_path))
print(files)
intputshape = 256
gth1 = np.zeros((48,intputshape,intputshape)).astype(int) 
for i,item in enumerate(results):
    gth = io.imread(os.path.join(gt_path,files[i]))
    gth = trans.resize(gth,(intputshape,intputshape))#512,512
    gth= np.array(((gth - np.min(gth))/np.ptp(gth))>0.1).astype(float)
    gth1[i,:,:] = gth

display = PrecisionRecallDisplay.from_predictions(gth1.flatten(),results[:,:,:,0].flatten(), name="LinearSVC")
_ = display.ax_.set_title("Precision-Recall curve")

Threshold = 0.5
results = np.where(results > Threshold, 1, 0)

TP = np.count_nonzero(results[:,:,:,0].flatten() * gth1.flatten())
TN = np.count_nonzero((results[:,:,:,0].flatten() - 1) * (gth1.flatten() - 1))
FP = np.count_nonzero(results[:,:,:,0].flatten() * (gth1.flatten() - 1))
FN = np.count_nonzero((results[:,:,:,0].flatten() - 1) * gth1.flatten())   
#print("tp = ",TP/(TP+FP)," Fp = ",FP/(TP+FP),"TN = ",TN/(TN+FN)," FN=",FN/(TN+FN))
try:
    precision = TP / (TP + FP)
except:
    precision = 0
recall = TP / (TP + FN)
Sensitivity = TP / (TP + FN)
Specificity = TN / (TN + FP)
DSC = 2* TP / (FP + 2 * TP+ FN)
f1 = 2 * precision * recall / (precision + recall)
accuracy = (TP + TN) / (TP + FP + TN + FN)
IoU=TP/(TP+FP+FN)
print("Precision=",precision, "Recall=",recall,"Sensitivity=" ,Sensitivity,"Specificity=",Specificity,"IoU=",IoU, "acc=",accuracy,"DSC=",DSC, "F1=",f1)
print("tp = ",TP/(TP+FP)," Fp = ",FP/(TP+FP),"TN = ",TN/(TN+FN)," FN=",FN/(TN+FN))

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
#fpr2, tpr2, _ = roc_curve(gth1.flatten(),results2[:,:,:,0].flatten())
fpr, tpr, _ = roc_curve(gth1.flatten(),results[:,:,:,0].flatten())
roc_auc = auc(fpr, tpr)
#roc_auc2 = auc(fpr2, tpr2)
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt

fig = plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (AUC = %0.2f)' % roc_auc)
#plt.plot(fpr2, tpr2, color='green', lw=lw, label='ROC curve2 (AUC = %0.2f)' % roc_auc2)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
#fig.savefig('/tmp/roc.png')
plt.show()